In [9]:
# ════════════════════════════════════════════════════════════════════
# CELLULE 1 : Imports et chargement des données
# ════════════════════════════════════════════════════════════════════

import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Charger les données
hotels = pd.read_csv('data/raw/hotels_top5_all.csv')

print(f"📊 Dataset chargé : {len(hotels)} hôtels")
print(f"📍 Villes : {hotels['city'].nunique()}")
print(f"\n🔍 Aperçu :")
print(hotels.info())

📊 Dataset chargé : 75 hôtels
📍 Villes : 5

🔍 Aperçu :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   hotel_id           75 non-null     object 
 1   city               75 non-null     object 
 2   hotel_name         75 non-null     object 
 3   url                75 non-null     object 
 4   score              75 non-null     float64
 5   number_of_reviews  75 non-null     int64  
 6   description        75 non-null     object 
 7   property_type      75 non-null     object 
 8   latitude           75 non-null     float64
 9   longitude          75 non-null     float64
 10  price              74 non-null     float64
 11  currency           74 non-null     object 
 12  facilities         75 non-null     object 
 13  image_url          75 non-null     object 
dtypes: float64(4), int64(1), object(9)
memory usage: 8.3+ KB
None


In [10]:
# ════════════════════════════════════════════════════════════════════
# CELLULE 2 : Analyse de la qualité des données
# ════════════════════════════════════════════════════════════════════

print("📊 QUALITÉ DES DONNÉES\n")
print("="*60)

# Valeurs manquantes
print("\n1. Valeurs manquantes :")
missing = hotels.isnull().sum()
missing_pct = (missing / len(hotels) * 100).round(1)
missing_df = pd.DataFrame({
    'Manquantes': missing,
    'Pourcentage': missing_pct
})
print(missing_df[missing_df['Manquantes'] > 0].sort_values('Manquantes', ascending=False))

# Doublons
print(f"\n2. Doublons détectés : {hotels.duplicated().sum()}")

# Distribution par ville
print("\n3. Hôtels par ville :")
print(hotels['city'].value_counts())

# Statistiques des prix
print("\n4. Prix :")
print(hotels['price'].describe())

# Statistiques des scores
print("\n5. Scores :")
print(hotels['score'].describe())

📊 QUALITÉ DES DONNÉES


1. Valeurs manquantes :
          Manquantes  Pourcentage
price              1          1.3
currency           1          1.3

2. Doublons détectés : 0

3. Hôtels par ville :
city
Aix en Provence       15
Marseille             15
Bormes les Mimosas    15
Cassis                15
Avignon               15
Name: count, dtype: int64

4. Prix :
count      74.000000
mean      329.054054
std       310.037333
min       105.000000
25%       175.000000
50%       249.500000
75%       323.750000
max      1998.000000
Name: price, dtype: float64

5. Scores :
count    75.00000
mean      8.33200
std       1.54484
min       0.00000
25%       8.00000
50%       8.50000
75%       9.10000
max       9.80000
Name: score, dtype: float64


In [11]:
# ════════════════════════════════════════════════════════════════════
# CELLULE 3 : Nettoyage des données
# ════════════════════════════════════════════════════════════════════

print("🧹 NETTOYAGE\n")
print("="*60)

# Copie de travail
hotels_clean = hotels.copy()

# 1. Supprimer les doublons exacts
duplicates = hotels_clean.duplicated(subset=['hotel_name', 'city']).sum()
if duplicates > 0:
    hotels_clean = hotels_clean.drop_duplicates(subset=['hotel_name', 'city'])
    print(f"✅ {duplicates} doublons supprimés")

# 2. Supprimer les hôtels sans nom ou URL
before = len(hotels_clean)
hotels_clean = hotels_clean.dropna(subset=['hotel_name', 'url'])
removed = before - len(hotels_clean)
if removed > 0:
    print(f"✅ {removed} hôtels sans nom/URL supprimés")

# 3. Gérer les prix manquants (optionnel : imputation par médiane)
missing_prices = hotels_clean['price'].isnull().sum()
if missing_prices > 0:
    print(f"⚠️  {missing_prices} hôtels sans prix (conservés)")

# 4. Gérer les scores manquants
missing_scores = hotels_clean['score'].isnull().sum()
if missing_scores > 0:
    print(f"⚠️  {missing_scores} hôtels sans score (conservés)")

# 5. Valider les coordonnées GPS (France : lat 41-51, lon -5 à 10)
invalid_coords = hotels_clean[
    (hotels_clean['latitude'].notna()) & 
    ((hotels_clean['latitude'] < 41) | (hotels_clean['latitude'] > 51) |
     (hotels_clean['longitude'] < -5) | (hotels_clean['longitude'] > 10))
]
if len(invalid_coords) > 0:
    print(f"⚠️  {len(invalid_coords)} coordonnées GPS invalides détectées")
    hotels_clean.loc[invalid_coords.index, ['latitude', 'longitude']] = np.nan

print(f"\n✅ Dataset nettoyé : {len(hotels_clean)} hôtels")

🧹 NETTOYAGE

✅ 1 doublons supprimés
⚠️  1 hôtels sans prix (conservés)

✅ Dataset nettoyé : 74 hôtels


In [12]:
# ════════════════════════════════════════════════════════════════════
# CELLULE 4 : Enrichissement des données
# ════════════════════════════════════════════════════════════════════

print("\n🎨 ENRICHISSEMENT\n")
print("="*60)

# 1. Catégories de prix
def categorize_price(price):
    if pd.isna(price):
        return 'Unknown'
    elif price < 80:
        return 'Budget'
    elif price < 150:
        return 'Moyen'
    else:
        return 'Luxe'

hotels_clean['price_category'] = hotels_clean['price'].apply(categorize_price)
print("✅ Catégories de prix créées")
print(hotels_clean['price_category'].value_counts())

# 2. Score pondéré (score × log(nombre d'avis))
hotels_clean['weighted_score'] = hotels_clean.apply(
    lambda row: row['score'] * np.log1p(row['number_of_reviews']) 
    if pd.notna(row['score']) and pd.notna(row['number_of_reviews']) 
    else np.nan,
    axis=1
)
print("\n✅ Score pondéré calculé")

# 3. Indicateur de disponibilité des données
hotels_clean['has_price'] = hotels_clean['price'].notna()
hotels_clean['has_gps'] = hotels_clean['latitude'].notna()
hotels_clean['has_score'] = hotels_clean['score'].notna()
hotels_clean['data_completeness'] = (
    hotels_clean['has_price'].astype(int) + 
    hotels_clean['has_gps'].astype(int) + 
    hotels_clean['has_score'].astype(int)
) / 3 * 100

print(f"\n📊 Complétude moyenne des données : {hotels_clean['data_completeness'].mean():.1f}%")


🎨 ENRICHISSEMENT

✅ Catégories de prix créées
price_category
Luxe       65
Moyen       8
Unknown     1
Name: count, dtype: int64

✅ Score pondéré calculé

📊 Complétude moyenne des données : 99.5%


In [14]:
# ════════════════════════════════════════════════════════════════════
# CELLULE 5 : Export des données nettoyées
# ════════════════════════════════════════════════════════════════════

output_path = '../data/processed/hotels_cleaned.csv'
hotels_clean.to_csv(output_path, index=False, encoding='utf-8')

print(f"\n💾 EXPORT\n")
print("="*60)
print(f"✅ Fichier sauvegardé : {output_path}")
print(f"📊 {len(hotels_clean)} hôtels")
print(f"📍 {hotels_clean['city'].nunique()} villes")
print(f"\n🔍 Aperçu des colonnes ajoutées :")
print(hotels_clean[['hotel_name', 'city', 'price', 'price_category', 
                     'score', 'weighted_score', 'data_completeness']].head())


💾 EXPORT

✅ Fichier sauvegardé : ../data/processed/hotels_cleaned.csv
📊 74 hôtels
📍 5 villes

🔍 Aperçu des colonnes ajoutées :
                                     hotel_name             city  price  \
0      Aparthotel Adagio Aix-en-Provence Centre  Aix en Provence  221.0   
1  Best Western Hotel le Galice Aix-en-Provence  Aix en Provence  233.0   
2       B&B HOTEL Aix-en-Provence Pont de l'Arc  Aix en Provence  128.0   
3                              Villa Saint-Ange  Aix en Provence  752.0   
4                                   Le Concorde  Aix en Provence  260.0   

  price_category  score  weighted_score  data_completeness  
0           Luxe    8.5       64.247392              100.0  
1           Luxe    8.0       63.022211              100.0  
2          Moyen    7.5       57.601317              100.0  
3           Luxe    9.3       55.020679              100.0  
4           Luxe    8.0       61.813229              100.0  


In [15]:
# ════════════════════════════════════════════════════════════════════
# CELLULE 6 : Statistiques finales
# ════════════════════════════════════════════════════════════════════

print(f"\n📊 STATISTIQUES FINALES\n")
print("="*60)

# Par ville
stats_by_city = hotels_clean.groupby('city').agg({
    'hotel_name': 'count',
    'price': 'mean',
    'score': 'mean',
    'data_completeness': 'mean'
}).round(2)

stats_by_city.columns = ['Nb hôtels', 'Prix moyen', 'Score moyen', 'Complétude (%)']
print("\n📍 Par ville :")
print(stats_by_city)

# Top 10 hôtels (score pondéré)
print("\n🏆 Top 10 hôtels (score pondéré) :")
top10 = hotels_clean.nlargest(10, 'weighted_score')[
    ['hotel_name', 'city', 'score', 'number_of_reviews', 'weighted_score', 'price']
]
print(top10)


📊 STATISTIQUES FINALES


📍 Par ville :
                    Nb hôtels  Prix moyen  Score moyen  Complétude (%)
city                                                                  
Aix en Provence            15      292.53         8.54          100.00
Avignon                    15      257.00         8.63          100.00
Bormes les Mimosas         15      445.21         8.11           97.78
Cassis                     15      372.73         8.09          100.00
Marseille                  14      295.29         8.30          100.00

🏆 Top 10 hôtels (score pondéré) :
                                           hotel_name             city  score  \
27                            NH Collection Marseille        Marseille    8.8   
0            Aparthotel Adagio Aix-en-Provence Centre  Aix en Provence    8.5   
1        Best Western Hotel le Galice Aix-en-Provence  Aix en Provence    8.0   
6                                      Hotel Cardinal  Aix en Provence    8.6   
7   Campanile Prime - A